## Predição do Agravamento do Quadro Clínico de Covid-19

* Analisa a probabilidade de agravamento a partir da necessidade de Suporte Ventilatório (Respirador);
* Utiliza dados de pacientes de Março de 2020 - Janeiro de 2021

In [3]:
# Para a correta execução do notebook será necessário instalar a biblioteca PyCaret
# pip install pycaret
# pip install openpyxl (Caso ocorra problema na leitura da base de dados)
# Tutorial: https://www.youtube.com/watch?v=88jD1nnkNfM&ab_channel=Let%27sData
# Site: https://www.learndatasci.com/tutorials/introduction-pycaret-machine-learning/
# !pip install pandas

!pip install openpyxl
!pip install pycaret

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from pycaret.classification import *

In [34]:
covidData = pd.read_excel("../database/dados-preprocessados-novos-organizados.xlsx")
#covidData

In [35]:
columnsWith9 = []
dataColumns = list(covidData.columns)

for i in range (0, len(dataColumns)):
    print(covidData[dataColumns[i]].value_counts())
    if 9 in covidData[dataColumns[i]].value_counts():
        columnsWith9.append(dataColumns[i])

1    470
0    304
Name: CS_SEXO, dtype: int64
5    216
4    204
6    160
3    130
7     30
2     26
0      6
1      2
Name: NU_IDADE_N, dtype: int64
4    535
1    201
2     21
9      9
3      8
Name: CS_RACA, dtype: int64
2    741
1     19
9     14
Name: SURTO_SG, dtype: int64
1    516
2    145
9    113
Name: FEBRE, dtype: int64
1    574
2    112
9     88
Name: TOSSE, dtype: int64
2    369
9    269
1    136
Name: GARGANTA, dtype: int64
1    647
2     66
9     61
Name: DISPNEIA, dtype: int64
1    604
9     88
2     82
Name: DESC_RESP, dtype: int64
1    523
9    128
2    123
Name: SATURACAO, dtype: int64
2    372
9    294
1    108
Name: DIARREIA, dtype: int64
2    402
9    303
1     69
Name: VOMITO, dtype: int64
1    557
2    217
Name: FATOR_RISC, dtype: int64
2    772
1      2
Name: PUERPERA, dtype: int64
2    536
1    238
Name: CARDIOPATI, dtype: int64
2    773
1      1
Name: HEMATOLOGI, dtype: int64
2    773
1      1
Name: SIND_DOWN, dtype: int64
2    758
1     16
Name: HEPATICA, dtyp

In [36]:
columnsWith9.remove('SUPORT_VEN')
columnsWith9.remove('CS_RACA')
columnsWith9

['SURTO_SG',
 'FEBRE',
 'TOSSE',
 'GARGANTA',
 'DISPNEIA',
 'DESC_RESP',
 'SATURACAO',
 'DIARREIA',
 'VOMITO',
 'UTI']

In [37]:
# Substituindo os valores 9 por 2 (Não apresentou)
for i in range (0, len(columnsWith9)):
    item = columnsWith9[i]
    covidData[item] = covidData[item].map({1:1, 2:2, 9:2})
    
covidData

,CS_SEXO,NU_IDADE_N,CS_RACA,SURTO_SG,FEBRE,TOSSE,GARGANTA,DISPNEIA,DESC_RESP,SATURACAO,DIARREIA,VOMITO,FATOR_RISC,PUERPERA,CARDIOPATI,HEMATOLOGI,SIND_DOWN,HEPATICA,ASMA,DIABETES,NEUROLOGIC,PNEUMOPATI,IMUNODEPRE,RENAL,OBESIDADE,UTI,SUPORT_VEN,EVOLUCAO,QNT_SEMANAS
0,0,4,4,2,1,1,1,1,1,1,2,2,1,2,1,2,2,2,2,2,2,2,2,2,2,1,2,1,4
1,1,5,4,2,1,1,1,1,1,1,2,2,1,2,2,2,2,2,2,1,2,2,2,2,2,2,2,1,1
2,1,4,4,2,1,1,2,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,2,1
3,1,4,4,2,1,1,2,1,1,1,2,2,1,2,2,2,2,2,2,1,2,2,2,2,2,1,1,2,2
4,1,5,4,2,1,1,2,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
769,1,3,4,2,2,1,2,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,0
770,0,6,1,2,1,2,2,2,1,1,1,2,1,2,1,2,2,2,2,1,2,2,2,2,2,1,2,1,2
771,0,5,4,2,1,1,2,1,1,1,2,2,1,2,2,2,2,2,2,1,2,2,2,2,2,1,2,2,3
772,1,4,4,2,2,2,2,1,1,1,2,2,1,2,1,2,2,2,2,2,2,2,2,2,2,2,2,1,1


In [39]:
covidData['SUPORT_VEN'] = covidData['SUPORT_VEN'].map({1:1, 2:1, 3: 2, 9:2})
for i in range (0, len(dataColumns)):
    print(covidData[dataColumns[i]].value_counts())

1    470
0    304
Name: CS_SEXO, dtype: int64
5    216
4    204
6    160
3    130
7     30
2     26
0      6
1      2
Name: NU_IDADE_N, dtype: int64
4    535
1    201
2     21
9      9
3      8
Name: CS_RACA, dtype: int64
2    755
1     19
Name: SURTO_SG, dtype: int64
1    516
2    258
Name: FEBRE, dtype: int64
1    574
2    200
Name: TOSSE, dtype: int64
2    638
1    136
Name: GARGANTA, dtype: int64
1    647
2    127
Name: DISPNEIA, dtype: int64
1    604
2    170
Name: DESC_RESP, dtype: int64
1    523
2    251
Name: SATURACAO, dtype: int64
2    666
1    108
Name: DIARREIA, dtype: int64
2    705
1     69
Name: VOMITO, dtype: int64
1    557
2    217
Name: FATOR_RISC, dtype: int64
2    772
1      2
Name: PUERPERA, dtype: int64
2    536
1    238
Name: CARDIOPATI, dtype: int64
2    773
1      1
Name: HEMATOLOGI, dtype: int64
2    773
1      1
Name: SIND_DOWN, dtype: int64
2    758
1     16
Name: HEPATICA, dtype: int64
2    758
1     16
Name: ASMA, dtype: int64
2    521
1    253
Name: DIABE

In [42]:
formatColumns = dataColumns
formatColumns.remove('NU_IDADE_N')
formatColumns.remove('CS_RACA')
formatColumns.remove('QNT_SEMANAS')
formatColumns

['CS_SEXO',
 'SURTO_SG',
 'FEBRE',
 'TOSSE',
 'GARGANTA',
 'DISPNEIA',
 'DESC_RESP',
 'SATURACAO',
 'DIARREIA',
 'VOMITO',
 'FATOR_RISC',
 'PUERPERA',
 'CARDIOPATI',
 'HEMATOLOGI',
 'SIND_DOWN',
 'HEPATICA',
 'ASMA',
 'DIABETES',
 'NEUROLOGIC',
 'PNEUMOPATI',
 'IMUNODEPRE',
 'RENAL',
 'OBESIDADE',
 'UTI',
 'SUPORT_VEN',
 'EVOLUCAO']

In [43]:
for i in range (0, len(formatColumns)):
    item = formatColumns[i]
    covidData[item] = covidData[item].map({2:0, 1:1})
    
for i in range (0, len(dataColumns)):
    print(covidData[dataColumns[i]].value_counts())

1.0    470
Name: CS_SEXO, dtype: int64
0    755
1     19
Name: SURTO_SG, dtype: int64
1    516
0    258
Name: FEBRE, dtype: int64
1    574
0    200
Name: TOSSE, dtype: int64
0    638
1    136
Name: GARGANTA, dtype: int64
1    647
0    127
Name: DISPNEIA, dtype: int64
1    604
0    170
Name: DESC_RESP, dtype: int64
1    523
0    251
Name: SATURACAO, dtype: int64
0    666
1    108
Name: DIARREIA, dtype: int64
0    705
1     69
Name: VOMITO, dtype: int64
1    557
0    217
Name: FATOR_RISC, dtype: int64
0    772
1      2
Name: PUERPERA, dtype: int64
0    536
1    238
Name: CARDIOPATI, dtype: int64
0    773
1      1
Name: HEMATOLOGI, dtype: int64
0    773
1      1
Name: SIND_DOWN, dtype: int64
0    758
1     16
Name: HEPATICA, dtype: int64
0    758
1     16
Name: ASMA, dtype: int64
0    521
1    253
Name: DIABETES, dtype: int64
0    747
1     27
Name: NEUROLOGIC, dtype: int64
0    744
1     30
Name: PNEUMOPATI, dtype: int64
0    762
1     12
Name: IMUNODEPRE, dtype: int64
0    741
1     33


In [ ]:
write = pd.ExcelWriter('dados-preprocessados2021-semIgnorados.xlsx')
covidData.to_excel(write, 'novosdados', index = False)
write.save()

In [43]:
attributes = ["CS_SEXO", "NU_IDADE_N", "CS_RACA", "FEBRE", "TOSSE", "GARGANTA", "DISPNEIA", "DESC_RESP", 
              "SATURACAO", "DIARREIA", "VOMITO", "FATOR_RISC", "CARDIOPATI", "DIABETES", "OBESIDADE", "SUPORT_VEN", "UTI", "QNT_SEMANAS", "EVOLUCAO"]

dadosReduzidos = covidData[attributes]
dadosReduzidos

,CS_SEXO,NU_IDADE_N,CS_RACA,FEBRE,TOSSE,GARGANTA,DISPNEIA,DESC_RESP,SATURACAO,DIARREIA,VOMITO,FATOR_RISC,CARDIOPATI,DIABETES,OBESIDADE,SUPORT_VEN,UTI,QNT_SEMANAS,EVOLUCAO
0,0,4,4,1,1,1,1,1,1,2,2,1,1,2,2,2,1,4,1
1,1,5,4,1,1,1,1,1,1,2,2,1,2,1,2,2,2,1,1
2,1,4,4,1,1,2,1,1,1,1,2,2,2,2,2,1,2,1,2
3,1,4,4,1,1,2,1,1,1,2,2,1,2,1,2,1,1,2,2
4,1,5,4,1,1,2,1,1,1,2,2,2,2,2,2,2,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
769,1,3,4,2,1,2,1,1,1,2,2,2,2,2,2,2,2,0,1
770,0,6,1,1,2,2,2,1,1,1,2,1,1,1,2,2,1,2,1
771,0,5,4,1,1,9,1,1,1,9,9,1,2,1,2,2,1,3,2
772,1,4,4,9,9,9,1,1,1,9,9,1,1,2,2,2,2,1,1


In [39]:
#for i in range(0, 16):
print(attributes[17], dadosReduzidos[attributes[17]].unique())

UTI [1 2 9]
